<h1>Creación de cadenas de ADN</h1>
<p>A partir del nucleótido anterior (A-> Adenina, C-> Citosina, G-> Guanina, T-> Timina) sacar el el siguiente nucleótido a partir de sus probabilidades dadas, que podemos representar en un circuito cuántico de la siguiente manera:
    
- |A> -> 0.2|A>+0.3|C>+0.3|G>+0.2|T>
- |C> -> 0.1|A>+0.41|C>+0.39|G>+0.1|T>
- |G> -> 0,25|A>+0.25|C>+0.25|G>+0.25|T>
- |T> -> 0.5|A>+0.17|C>+0.17|G>+0.17|T>
    
En este circuito vamos a declarar A -> 00 , C -> 01 , G -> 10, T -> 11
</p>

In [1]:
from qiskit import *
from qiskit.visualization import plot_histogram
import numpy as np

#INPUTS
tripletes=10
phi ="00" #Primer nucleótido. Únicos valores permitidos: 00,01,10,11
n_cadenas=1

In [2]:
def circA(): #0.2|A>+0.3|C>+0.3|G>+0.2|T>
    circ = QuantumCircuit(2)
    circ.h(1)
    circ.crx(0.556*np.pi,1,0)
    circ.x(1)
    circ.crx(0.435*np.pi,1,0)
    gate = circ.to_gate()
    gate.name = "A"
    gate = gate.control(2)
    return gate

In [3]:
def circC():#0.1|A>+0.41|C>+0.39|G>+0.1|T>
    circ = QuantumCircuit(2)
    circ.h(1)
    circ.crx(0.705*np.pi,1,0)
    circ.x(1)
    circ.crx(0.3*np.pi,1,0)
    
    gate = circ.to_gate()
    gate.name = "C"
    gate = gate.control(2)
    return gate

In [4]:
def circG():#0,25|A>+0.25|C>+0.25|G>+0.25|T>
    circ = QuantumCircuit(2)
    circ.h(range(2))
    
    gate = circ.to_gate()
    gate.name = "G"
    gate = gate.control(2)
    return gate

In [5]:
def circT():#0.5|A>+0.17|C>+0.17|G>+0.17|T>
    circ = QuantumCircuit(2)
    circ.h(0)
    circ.crx(0.608*np.pi,0,1)
    circ.crx(np.pi/2,1,0) # o la puerta sqrt(x)
    
    gate = circ.to_gate()
    gate.name = "T"
    gate = gate.control(2)
    return gate

In [6]:
def bin_letra(bin):
    string = ""
    if bin == "00":
        string = "A"
    if bin == "01":
        string ="C"
    if bin == "10":
        string ="G"
    if bin == "11":
        string ="T"
    return string

In [7]:
def asignarDatos(phi):
    circ = QuantumCircuit(2)
    pos=0
    for i in phi:
        if i=="1":
            circ.x(pos)
        pos = pos+1
    gate = circ.to_gate()
    gate.name="Datos"
    return gate

In [8]:
#IMPORTANTE: Este circuito sigue teniendo en cuenta la probabilidad del qubit anterior, n
circ = QuantumCircuit(4,6*tripletes+1) #dará un error enorme si lo mando a un ordenador real (circuito demasiado grande). Mejor un circ con cambio de input (pero habría que modificarlo entre shots)
circ.append(asignarDatos(phi),(0,1))
circ.barrier(range(4))
for i in range(tripletes*3):
    circ.x(range(2))
    circ.append(circA(),range(4))
    circ.x(0)
    circ.append(circC(),range(4))
    circ.x(range(2))
    circ.append(circG(),range(4))
    circ.x(1)
    circ.append(circT(),range(4))
    circ.reset(0)
    circ.reset(1)
    circ.cx(2,0)
    circ.cx(3,1)
    circ.measure([2,3],[2*i,2*i+1])
    circ.reset(2)
    circ.reset(3)
    circ.barrier(range(4))

In [ ]:
backend = Aer.get_backend("qasm_simulator")
string = ""
job = execute(circ, backend, shots = n_cadenas, memory=True)
data = job.result().get_memory()
print(data)

In [ ]:
output = ""
for i in range(n_cadenas):
    for j in range(len(data[i])//2):
        binario = data[0][2*j:2*j+2]
        output = output + bin_letra(binario)
        if j%3==2:
            output = output + "-"
    print(output)